In [15]:
import sys
sys.path.insert(1,'/Users/charlesh/Documents/Codes/')
sys.path.append('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS')
sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS')
sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/ATaCR/ATaCR_Python/OBStools')
sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/ATaCR/ATaCR_Python')

import math
import numpy as np
import librosa
from obspy import Trace
import librosa.display
import matplotlib.pyplot as plt
import sys
import pandas as pd

sys.path.insert(0, '/Users/charlesh/Documents/Codes/ATaCR')
import ObsQA
# from ObsQA.plots import qtp
import obspy

# %cd /Users/charlesh/Documents/Codes/ATaCR/ATaCR_Comp
import glob as g
import pickle as pkl
import pandas as pd
import numpy as np
import scipy as sc
import scipy.stats as stats
import matplotlib.pyplot as plt
from IPython.display import clear_output, display #Only works in notebooks

from obspy.core import read, Stream, Trace, AttribDict, UTCDateTime
# obspy.geodetics.base.gps2dist_azimuth
from obspy.clients.fdsn import Client
import seaborn as sns
import datetime
import ObsQA as ob
import obstools as obs
# My Frankenstein of QA tools:

import folium
from branca.element import Figure
# from branca.element import Template, MacroElement
import geemap
import matplotlib
import distinctipy
import ee
import os
import io
from PIL import Image
import PIL
sys.path.insert(0, '/Users/charlesh/Documents/Codes/ATaCR/ATaCR_Comp/')
from comp_tools import *

import io

In [16]:
# Load catalog
NoiseFolder = '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE'
CompFolder = NoiseFolder + '/COMPS/ATaCR_NC'
MethodsFolder = NoiseFolder + '/METHODS/ATaCR'
compfolder = MethodsFolder
datafolder = ob.io.dir_libraries(MethodsFolder)[1]['Py_DataParentFolder']
ATaCR_Py_DataFolder = ob.io.dir_libraries(compfolder)[1]
eventsfolder = ATaCR_Py_DataFolder['Py_CorrectedTraces']
catalog = pd.read_pickle(eventsfolder + '/sta_catalog_proxima_test.pkl')
catalog['Network_Experiment'] = '[' + catalog.Network + '] ' + catalog.Experiment
elevation = ['black', 'afafaf', 'blue', 'white', 'green']
hexclrs = ['#ff0000','#7852fb','#38f830','#c90efd','#da0543','#0df9bf','#fa47ab','#14b5b5','#be0305']
Stations = ObsQA.io.getstalist()
# display(catalog)

In [17]:
evaudit = pd.read_pickle(Path(eventsfolder) / 'event_record_audit.pkl')
evsilike = ['2011.191.00.57','2010.163.19.26','2010.096.22.15']
evsi = [np.where(evaudit.Event==d)[0][0] for d in evsilike]
evaudit = evaudit.iloc[evsi]

## MAP

In [18]:
def map(color_by=None,stations_subset=None,stations=None,figsize=None,key=0,legend_title='Legend',legx='"auto"',legy='"auto"',hexclrs=None,legend_on=False,ev_lla=None,center=None,lobnds=None,zoom_start=1,color=None,ev_icon_size=(0,0),marker_icon_size=(0,0)):
        sta_set = None
        sta_subset = None
        if stations_subset is None and stations is None:
                sta_set,sta_subset = ObsQA.io.getstalist()
        if stations is not None:
                sta_set = stations.copy()
        if stations_subset is not None:
                sta_subset = stations_subset.copy()
        if not lobnds:
                if sta_set is not None:
                        labnds = [sta_set['Latitude'].min(),sta_set['Latitude'].max()]
                        lobnds = [sta_set['Longitude'].min(),sta_set['Longitude'].max()]
                if sta_subset is not None:
                        labnds = [sta_subset['Latitude'].min(),sta_subset['Latitude'].max()]
                        lobnds = [sta_subset['Longitude'].min(),sta_subset['Longitude'].max()]
        sta_set_orig = sta_set.copy()
        if color_by is not None:
                items = sta_set[color_by].unique()
                sta_set = [sta_set.iloc[np.in1d(sta_set[color_by],item)] for item in items]
        else:
                sta_set = [sta_set]
        tilekey = dict()
        # good map tiles: http://leaflet-extras.github.io/leaflet-providers/preview/
        tilekey[0] = ('https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}','Sources: GEBCO, NOAA, CHS, OSU, UNH, CSUMB, National Geographic')
        tilekey[1] = ('https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}','Sources: USGS, Esri, TANA, DeLorme, and NPS')
        tilekey[2] = ('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}','Sources: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community')
        if figsize is None:
                if sta_subset is not None:
                        figsize = [600,500]
                else:
                        figsize = [2000,800]
        fig = Figure(width=figsize[0], height=figsize[1])
        print('lobnds::')
        print(lobnds)
        if center is not None:
                print('Center::')
                print(center)
                m = folium.Map(location=center,tiles=tilekey[key][0],attr=tilekey[key][1],control=False,max_bounds=True,scrollWheelZoom=True,zoom_start=zoom_start,min_zoom=2,max_zoom=20) #,width=figsize[0],height=figsize[1]
        else:
                m = folium.Map(tiles=tilekey[key][0],attr=tilekey[key][1],zoom_start=1,min_zoom=1,minlon = lobnds[0],maxlon=lobnds[1],control=False,max_bounds=False,zoom_control=False,scrollWheelZoom=True,max_zoom=20) #,width=figsize[0],height=figsize[1]
        m.add_child(folium.LatLngPopup())
        
        # lat_interval = 10
        # lon_interval = 10
        # for lat in range(0, 360, lat_interval):
        #         folium.PolyLine([[lat, -180],[lat, 180]], weight=0.5).add_to(m)
        # for lon in range(0,360, lon_interval):
        #         folium.PolyLine([[-90, lon],[90, lon]], weight=0.5).add_to(m)

        if hexclrs is None:
                if len(sta_set)==1:
                        hexclrs = ['#e41a1c'] #red
                else:
                        colors = distinctipy.get_colors(len(sta_set),pastel_factor=0.0,exclude_colors=[(1.0, 0.0, 1.0),(1,1,1),(0,0,0),(0,1,0),(0,1,1)],colorblind_type='Deuteranomaly',n_attempts=1000)
                        hexclrs = [matplotlib.colors.to_hex(colors[i]) for i in range(len(colors))][0:len(colors)]
        hexclrs = hexclrs[0:len(sta_set)]
        if color_by is None:
                color_by = 'Station'
        if sta_set is not None:
                legend_dict = dict()
                for clri,cur_sta_set in enumerate(sta_set):
                        if not color:
                                color = hexclrs[clri]
                        print('plotting sta_set')
                        label = str(cur_sta_set[color_by].tolist()[0])
                        legend_dict[label]  = color
                        locationlist = (np.array((cur_sta_set['Latitude'],list(cur_sta_set['Longitude']))).T.tolist())
                        g1 = folium.FeatureGroup(name=('Stations').format( txt='All Stations'),keep_in_front=True)
                        for point in range(0, len(locationlist)):
                                # icon = folium.Icon(color='white',icon_color=color,prefix='fa',icon='caret-up',shadow_size=0,icon_size=marker_icon_size,icon_anchor=(0,0))
                                # marker = folium.Marker(locationlist[point], popup=list(cur_sta_set['Network'] + '.' + list(cur_sta_set['Station']) + ' (' + list(cur_sta_set['Experiment']) + ')')[point],icon=folium.Icon(color='white',icon_color=color,prefix='fa',icon='caret-up',shadow_size=0,icon_size=(0,0),icon_anchor=(0,0)),legend_name='hello')
                                LatLon = locationlist[point]                                
                                marker = folium.Marker(LatLon,icon=folium.Icon(color='white',icon_color='blue',prefix='fa',icon='caret-up',shadow_size=0,icon_size=marker_icon_size,icon_anchor=(0,0)),legend_name='hello',opacity=1)
                                marker.style = {'transform': 'scale(6000)'}
                                # L = folium.FeatureGroup().add_child(marker)
                                marker.add_to(g1)
                                
                                # marker = folium.Marker([locationlist[point][0],locationlist[point][1]-360],icon=icon,legend_name='hello')
                                # L = folium.FeatureGroup().add_child(marker)
                                # marker.add_to(g1)
                        m.add_child(g1)

                if legend_on:
                        if len(sta_set)==1:
                                labels = [color_by]
                        else:
                                labels = list(legend_dict.keys())
                        colors = list(legend_dict.values())
                        macro = ObsQA.getLegendTemplate(colors,labels,title_str=legend_title,legx=legx,legy=legy)
                        m.add_child(macro)

        if ev_lla is not None:
                g3 = folium.FeatureGroup(name=('Events').format(txt='Subset'),keep_in_front=True)
                for ev in ev_lla:
                        loc = (np.array(([ev[0]],[ev[1]])).T.tolist())
                        # icon = folium.Icon(color='white',icon_color='black',prefix='fa',icon='star',shadow_size=0,icon_size=ev_icon_size,icon_anchor=(0,0))
                        marker = folium.Marker(loc[0],icon=folium.Icon(color='white',icon_color='red',prefix='fa',icon='star',shadow_size=0,icon_size=ev_icon_size,icon_anchor=(0,0)),keep_in_front=True,opacity=1)
                        marker.style = {'transform': 'scale(6000.0)'}
                        marker.add_to(g3)
                m.add_child(g3)
        # sta_lla = sta_subset[['Latitude','Longitude']]
        
        display(sta_subset)
        sta_lla = [sta_set_orig[['Latitude', 'Longitude']].min().values.tolist()]
        if ev_lla:
                [sta_lla.extend([e]) for e in ev_lla]
        sta_lla = np.array(sta_lla)
        if sta_set is not None:
                fit_bounds=[sta_set_orig[['Latitude', 'Longitude']].min().values.tolist(), sta_set_orig[['Latitude', 'Longitude']].max().values.tolist()]
                fit_bounds = [sta_lla.min(axis=0).tolist(), sta_lla.max(axis=0).tolist()]
                m.fit_bounds(fit_bounds)
                print('Bounds fit::')
                print(fit_bounds)
        fig.add_child(m)
        return m,fig

# ##---------------------------------------------------------------------------------------
# ##-------------------------------------------------------ARRAY MAP PLOTS-----------------
# ##---------------------------------------------------------------------------------------
evaudit = pd.read_pickle(Path(eventsfolder) / 'event_record_audit.pkl')
evsilike = ['2011.191.00.57','2010.163.19.26','2010.096.22.15']
evsi = [np.where(evaudit.Event==d)[0][0] for d in evsilike]
evaudit = evaudit.iloc[evsi]

# catalog = Stations
save_folder = '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS/FigureArchive/_GEN5/EventRecords/SSA_24_Poster_Charles/some_maps'
catalog['Network_Experiment'] = '[' + catalog.Network + '] ' + catalog.Experiment
scl = 2
figsize = [scl*600,scl*300]
fields = ['Network_Experiment']
# fields = [fields[0]]
# distinctipy.color_swatch(distinctipy.get_colors(100),show_text=True)
cat = catalog.copy()
# evaudit = evaudit.iloc[0].to_frame().T
center = [21.2894,-152.8984]
center = None
zoom_start = 2
color = 'black'
color = None
marker_icon_size = (0,0)
ev_icon_size = (0,0)
# marker_icon_size = (50,50)
# ev_icon_size = (50,50)
for ev in evaudit.iloc:
        ev_lla = [ev.ev_lla,abs(ev.ev_lon)-360]
        # ev_lla = None
        stanames = [a + '.' + b for a,b in zip(ev.Networks,ev.Stations)]
        current_cat = cat.copy().iloc[[np.where(cat.StaName==s)[0][0] for s in stanames]]
        for color_by in fields:
                display(color_by)
                m,fig = map(color_by=None,stations=current_cat,figsize = figsize,legend_title=color_by,legx='300px',legy='120px',hexclrs=None,key=1,ev_lla=[ev_lla],center=center,zoom_start=zoom_start,color=color,marker_icon_size=marker_icon_size,ev_icon_size=ev_icon_size)
                display(m)
                img_data = m._to_png(8)
                img = Image.open(io.BytesIO(img_data))
                fname = 'StaMap_e' + ev.Event
                # img = img.crop(box=(0,0,figsize[0],figsize[1]))
                file = Path(save_folder) / (fname + '.png')
                img.save(file,quality=100, subsampling=0,dpi=(12000, 1200),optimize=True)
                file = Path(save_folder) / (fname + '.html')
                m.save(file)
                print('done')

'Network_Experiment'

lobnds::
[-124.647202, -118.480202]
plotting sta_set


None

Bounds fit::
[[32.777, -216.6984], [38.0553, -124.647202]]


done


...

'Network_Experiment'

lobnds::
[-178.155304, 155.108505]
plotting sta_set


None

Bounds fit::
[[-21.951799, -262.8887], [2.3601, -178.155304]]


done


In [ ]:
evaudit = pd.read_pickle(Path(eventsfolder) / 'event_record_audit.pkl')
evsilike = ['2011.191.00.57','2010.163.19.26','2010.096.22.15']
evsi = [np.where(evaudit.Event==d)[0][0] for d in evsilike]
evaudit = evaudit.iloc[evsi]
evaudit

In [ ]:
[display(catalog[catalog.Network==N].iloc[0].to_frame().T) for N in ['2D','YL','ZN','Z6']]

In [ ]:
# # ##---------------------------------------------------------------------------------------
# # ##-------------------------------------------------------ARRAY MAP PLOTS-----------------
# # ##---------------------------------------------------------------------------------------
# # catalog = Stations
# catalog['Network_Experiment'] = '[' + catalog.Network + '] ' + catalog.Experiment
# figsize = [1500,600]
# fields = ['Network_Experiment','Seismometer','Pressure_Gauge','Instrument_Design','n_events']
# # fields = [fields[0]]
# # distinctipy.color_swatch(distinctipy.get_colors(100),show_text=True)
# for color_by in fields:
#     display(color_by)
#     m = map(color_by=color_by,stations=catalog,figsize = figsize,legend_title=color_by,legx='300px',legy='120px',hexclrs=hexclrs,key=1)
#     display(m)
#     img_data = m._to_png(4)
#     img = Image.open(io.BytesIO(img_data))
#     file = '/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/report_11_15/maps/Everything_MapBy_' + color_by + '.png'
#     img = img.crop(box=(0,0,500,600))
#     img.save(file,quality=100, subsampling=0)

In [ ]:
# ##---------------------------------------------------------------------------------------
# ##-------------------------------------------------------DAY NOISE SUPPLY PLOT-----------
# ##---------------------------------------------------------------------------------------

# # catalog_audit = ObsQA.io.AuditEventFolder(ATaCR_Py_DataFolder['Py_CorrectedTraces'])
# catalog_audit = AuditNoiseSupply(catalog_audit)
# PreEventDaySupply_Mean = np.round([d.mean() for d in catalog_audit.PreEventDaySupply.iloc])
# PreEventDaySupply_Min = np.array([d.min() for d in catalog_audit.PreEventDaySupply.iloc])
# barx = catalog_audit.Network + '.' + catalog_audit.Station
# sex_counts = {'Mean': PreEventDaySupply_Mean,'Min': PreEventDaySupply_Min}
# width = 0.8  # the width of the bars: can also be len(x) sequence
# fig, ax = plt.subplots(figsize=(15,3))
# bottom = np.zeros(len(barx))
# c = ['r','b']
# i = 0
# for ylbl, yv in sex_counts.items():
#     p = ax.bar(barx, yv, width, label=ylbl, bottom=bottom,color=c[i])
#     i+=1
# plt.xticks(rotation=40,fontweight='bold')
# plt.grid('on')
# ax.set_title('Event Day Supply',fontweight='bold')
# ax.set_xlim(0-4*width,len(barx))
# ax.legend(loc='center left')
# fig.tight_layout()
# plt.show()
# fig.savefig('/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/report_11_15/DaySupply.png')

In [ ]:
# ##---------------------------------------------------------------------------------------
# ##-------------------------------------------------------RAY COVERAGE PLOTS--------------
# ##---------------------------------------------------------------------------------------

# cat_by_net = [catalog[np.in1d(catalog.Network,c)] for c in catalog.Network.unique()]
# bazdist = dict()
# for cat in cat_by_net:
#         BAZ = []
#         DIST = []
#         HIST = []
#         STAZ = []
#         for sta in cat.iloc:
#                 for ev in sta.Metadata:
#                         stalla = [sta.Latitude,sta.Longitude]
#                         evlla = [ev[0].origins[0].latitude,ev[0].origins[0].longitude]
#                         dist,baz,az = obspy.geodetics.base.gps2dist_azimuth(stalla[0],stalla[1],evlla[0],evlla[1])
#                         BAZ.append(baz)
#                         DIST.append(dist/1000)
#                         STAZ.append(sta.Water_Depth_m)
#         bazdist[cat.Network.iloc[0]] = [BAZ,DIST,HIST,STAZ]
# for key in list(bazdist.keys()):
#         counts, bins = np.histogram(bazdist[key][0])
#         bazdist[key][2] = [counts[np.where(counts>0)],bins[np.where(counts>0)]]
# fig, ax = plt.subplots(nrows=2,ncols=len(list(bazdist.keys())),figsize=(24,10),subplot_kw={'projection': 'polar'})
# width = 0.15
# labels = '[' + catalog.Network.unique() + '] ' + catalog.Experiment.unique()
# for i,net in enumerate(list(bazdist.keys())):
#         bz = bazdist[net][0]
#         dst = bazdist[net][1]
#         cts_cts = bazdist[net][2][0]
#         cts_bz = bazdist[net][2][0]
#         staz = bazdist[net][3]
#         ax[0,i].bar(bz, dst, width=width, bottom=0.0,alpha=0.9,facecolor=hexclrs[i],label=labels[i],edgecolor='k')
#         ax[0,i].legend(loc='upper center')
#         ax[1,i].bar(bz, staz, width=width, bottom=0.0,alpha=0.9,facecolor=hexclrs[i],label=labels[i],edgecolor='k')
#         ax[1,i].legend(loc='upper center')
# ax[0,2].set_title('Event Back-Azimuth & Event Depth (km)',fontweight='bold')
# ax[1,2].set_title('Event Back-Azimuth & Station Depth (m)',fontweight='bold')
# plt.show()
# # fig.savefig('/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/report_11_15/RayCover.png')
# save_tight('/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/SubArray34n/RayCover.png',fig)

In [ ]:
# m.save(Path(save_folder) / (fname + '.png'))
# Path(save_folder) / (fname + '.png')

# import os
# import time
# from selenium import webdriver

delay=5
fn='testmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

browser = webdriver.Firefox()
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot(Path(save_folder) / (fname + '.pdf'))
browser.quit()

In [ ]:
# ##---------------------------------------------------------------------------------------
# ##-------------------------------------------------------METADATA PLOTS------------------
# ##---------------------------------------------------------------------------------------
# catalog = Stations
# # colors = distinctipy.get_colors(100)
# # colors = hexclrs
# catalog['Network_Experiment'] = '[' + catalog.Network + '] ' + catalog.Experiment
# filter_sets = ['Network_Experiment','Pressure_Gauge','Instrument_Design','Seismometer']
# x_by = ['Water_Depth_m','Station Depth (m)']
# y1_by = ['Surface_Current_ms','Surface Current (cm/s)']
# y3_by = ['Distance_from_Land_km','Distance from Land (km)']
# y2_by = ['Sediment_Thickness_m','Sediment Thickness (m)']
# fig,ax  = plt.subplots(nrows=len(filter_sets),ncols=3,figsize=(20,15),sharex='col',sharey='row')
# s = 100
# fnt = 14
# for rw,filter_by in enumerate(filter_sets):
#     ax = np.atleast_2d(ax)
#     cat_by_net = [catalog[np.in1d(catalog[filter_by],net)] for net in catalog[filter_by].unique()]
#     for i,cat in enumerate(cat_by_net):
#         # if cat.Network.iloc[0]=='7D':
#         #     continue
#         XX,Y1,Y2,Y3 = [],[],[],[]
#         for sta in cat.iloc:
#             x = np.array(sta[x_by[0]])
#             XX.extend(x.reshape(-1).tolist())
#             Y1.extend(np.array(sta[y1_by[0]]).reshape(-1).tolist())
#             Y2.extend(np.array(sta[y2_by[0]]).reshape(-1).repeat(x.size).tolist())
#             Y3.extend(np.array(sta[y3_by[0]]).reshape(-1).repeat(x.size).tolist())
#         label = sta[filter_by]
#         Y1 = np.array(Y1)*100
#         ax[rw,0].scatter(Y1,XX,c=colors[i],label=label,s=s)
#         ax[rw,1].scatter(Y2,XX,c=colors[i],s=s,label=label)
#         ax[rw,2].scatter(Y3,XX,c=colors[i],label=label,s=s)
#         ax[rw,0].grid('on')
#         ax[rw,1].grid('on')
#         ax[rw,2].grid('on')
#         if rw>0:
#             lg = ax[rw,2].legend(loc='lower right',fontsize='small',prop=dict(weight='bold'),framealpha=1.0)
#             # for i in range(len(lg.legend_handles)):
#                 # lg.legend_handles[i]._sizes = np.array([20])
#                 # lg.legend_handles[i].fontsize = 6
#         ax[rw,0].set_ylabel(x_by[1],fontweight='bold',fontsize=fnt)
#         if rw==0:
#             ax[rw,0].set_title(y1_by[1],fontweight='bold',fontsize=fnt)
#             ax[rw,1].set_title(y2_by[1],fontweight='bold',fontsize=fnt)
#             ax[rw,2].set_title(y3_by[1],fontweight='bold',fontsize=fnt)
#         if rw==(len(filter_sets)-1):
#             ax[rw,0].set_xlabel(y1_by[1],fontweight='bold',fontsize=fnt)
#             ax[rw,1].set_xlabel(y2_by[1],fontweight='bold',fontsize=fnt)
#             ax[rw,2].set_xlabel(y3_by[1],fontweight='bold',fontsize=fnt)
# save_tight('/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/SubArray34n/test_MetaData.png',fig)

In [ ]:
lg.legend_handles[i]._s